In [2]:
import time
from math import ceil

from helium import *
from selenium.webdriver.common.by import By
import json

In [3]:
from helium import start_firefox

driver = start_firefox()

In [15]:
def load_breweries_base():
    with open('beer_db.json', encoding='utf-8') as f:
        return json.load(f)


def save_breweries_base(breweries):
    with open('beer_db.json', 'w', encoding='utf-8') as f:
        return json.dump(breweries, f, )


all_breweries = load_breweries_base()
print(len(all_breweries))


55


In [5]:
with open('new-breweries.txt', encoding='utf-8') as f:
    new_breweries = f.read().splitlines()
new_breweries[:10], len(new_breweries)

(['https://untappd.com/BrasseriedeSilly/beer',
  'https://untappd.com/w/asahi-breweries/50/beer',
  'https://untappd.com/RebelApple/beer',
  'https://untappd.com/youtonbrewery/beer',
  'https://untappd.com/NXbrew/beer',
  'https://untappd.com/paradox-beer/beer',
  'https://untappd.com/GurinCraft/beer',
  'https://untappd.com/BreweryGusi/beer',
  'https://untappd.com/KLAATUBREW/beer',
  'https://untappd.com/w/tridevyatoe/477859/beer'],
 82)

In [28]:
def scroll_till_the_end():
    for i in range(ceil(300 / 25) + 1):
        scroll_down(10000)
        
        try:
            load_more_btn = S('.more-list-items').web_element
        except LookupError:
            break
        else:
            load_more_btn.click()
            time.sleep(0.5)

In [7]:
def parse_brewery(url):
    brewery = {
        'title': S('.b_info .name h1').web_element.text,
        'location': S('.b_info .name .brewery').web_element.text,
        'url': url
    }

    # Extract beers information
    beers = []
    for beer_item in find_all(S('.beer-item')):
        beer_item = beer_item.web_element
        title = beer_item.find_element(By.CLASS_NAME, 'name').text.strip()
        link = beer_item.find_element(By.TAG_NAME, 'a').get_attribute('href').strip()
        style = beer_item.find_element(By.CLASS_NAME, 'style').text.strip()
        abv = beer_item.find_element(By.CLASS_NAME, 'abv').text.strip()
        ibu = beer_item.find_element(By.CLASS_NAME, 'ibu').text.strip()
        rating = beer_item.find_element(By.CSS_SELECTOR, '.num').text.strip()
        rating_amount = beer_item.find_element(By.CLASS_NAME, 'raters').text.strip()
        added = beer_item.find_element(By.CLASS_NAME, 'date').text.strip()

        # Extract ABV, IBU, rating, ratingAmount, and added
        abv = float(abv.split('%')[0]) if 'N/A' not in abv else 0
        added = added[6:] if 'Added' in added else added
        ibu = int(ibu.split(' ')[0]) if 'N/A' not in ibu else 0
        rating = float(rating[1:-1]) if 'N/A' not in rating else 0
        rating_amount = int(rating_amount.split(' ')[0].replace(',', ''))

        # Split style into mainStyle and subStyle
        main_style, sub_style = style.split(' - ') if ' - ' in style else (style, '')

        # Add beer to beers list
        beers.append({
            'title': title,
            'link': link,
            'style': style,
            'abv': abv,
            'ibu': ibu,
            'rating': rating,
            'ratingAmount': rating_amount,
            'added': added,
            'mainStyle': main_style,
            'subStyle': sub_style,
            'breweryLink': url
        })

    return dict(brewery=brewery, beers=beers)


In [24]:
url = new_breweries[3]
driver.execute_script(f'location.replace("{url}")')


In [26]:

brewery_dict = parse_brewery(url)
all_breweries.append(brewery_dict)
save_breweries_base(all_breweries)
print(f"parsed: {len(brewery_dict['beers'])}; all_breweries: {len(all_breweries)}")

parsed: 53; all_breweries: 58


In [29]:
for url in new_breweries[4:]:
    driver.execute_script(f'location.replace("{url}")')
    time.sleep(1)
    scroll_till_the_end()
    brewery_dict = parse_brewery(url)
    all_breweries.append(brewery_dict)
    save_breweries_base(all_breweries)
    print(f"parsed: {len(brewery_dict['beers'])}; all_breweries: {len(all_breweries)}")

parsed: 38; all_breweries: 59
parsed: 166; all_breweries: 60
parsed: 77; all_breweries: 61
parsed: 233; all_breweries: 62
parsed: 12; all_breweries: 63
parsed: 9; all_breweries: 64
parsed: 313; all_breweries: 65
parsed: 137; all_breweries: 66
parsed: 38; all_breweries: 67
parsed: 135; all_breweries: 68
parsed: 14; all_breweries: 69
parsed: 267; all_breweries: 70
parsed: 31; all_breweries: 71
parsed: 75; all_breweries: 72
parsed: 87; all_breweries: 73
parsed: 28; all_breweries: 74
parsed: 121; all_breweries: 75
parsed: 82; all_breweries: 76
parsed: 26; all_breweries: 77
parsed: 98; all_breweries: 78
parsed: 27; all_breweries: 79
parsed: 19; all_breweries: 80
parsed: 21; all_breweries: 81
parsed: 11; all_breweries: 82
parsed: 43; all_breweries: 83
parsed: 44; all_breweries: 84
parsed: 95; all_breweries: 85
parsed: 19; all_breweries: 86
parsed: 64; all_breweries: 87
parsed: 242; all_breweries: 88
parsed: 114; all_breweries: 89
parsed: 102; all_breweries: 90
parsed: 16; all_breweries: 91
p